In [14]:
import platform
import warnings
warnings.filterwarnings('ignore')

from os import listdir
from os.path import isfile, join
import math
import itertools
import random
import numpy as np
import pandas as pd
import networkx as nx
from networkx.algorithms.community.centrality import girvan_newman
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px

random.seed(200)
print('Python version: {}'.format(platform.python_version()))
print('NetworkX version: {}'.format(nx.__version__))
print('Matplotlib version: {}'.format(matplotlib.__version__))
print('Numpy version: {}'.format(np.__version__))

Python version: 3.7.4
NetworkX version: 2.4
Matplotlib version: 3.1.1
Numpy version: 1.18.1


In [3]:
mypath = 'raccoon_proximity_weighted'
files = [mypath + '/' + f for f in listdir(mypath) if isfile(join(mypath, f))]
files = [(int(f.split('_')[5].split('.')[0]), f) for f in files]
files.sort()
#files = [f[1] for f in files]

In [49]:
df = pd.DataFrame()
filt = [str(x) for x in range(16)]

for f in files:
    G = nx.read_graphml(f[1]).subgraph(filt)
    deg = pd.DataFrame({'week':f[0], 'raccoon_ID':list(G), 'degree':list(dict(G.degree).values())})
    DC = nx.degree_centrality(G)
    DC = pd.DataFrame({'week':f[0], 'raccoon_ID':list(DC.keys()),
                           'deg_cent':list(DC.values())})
    BC = nx.betweenness_centrality(G)
    BC = pd.DataFrame({'week':f[0], 'raccoon_ID':list(BC.keys()),
                           'bet_cent':list(BC.values())})
    CC = nx.closeness_centrality(G)
    CC = pd.DataFrame({'week':f[0], 'raccoon_ID':list(CC.keys()),
                           'clos_cent':list(CC.values())})
    EC = nx.eigenvector_centrality(G)
    EC = pd.DataFrame({'week':f[0], 'raccoon_ID':list(EC.keys()),
                           'eig_cent':list(EC.values())})
    
    df_aux = DC.merge(BC, how='outer', on=['week', 'raccoon_ID'])
    df_aux = df_aux.merge(CC, how='outer', on=['week', 'raccoon_ID'])
    df_aux = df_aux.merge(EC, how='outer', on=['week', 'raccoon_ID'])
    df_aux = df_aux.merge(deg, how='outer', on=['week', 'raccoon_ID'])
    df = pd.concat([df,df_aux])
df['raccoon_ID'] = pd.to_numeric(df['raccoon_ID'])
df

,week,raccoon_ID,deg_cent,bet_cent,clos_cent,eig_cent,degree
0,1,1,0.142857,0.000000,0.388889,0.096298,2
1,1,2,0.285714,0.031136,0.451613,0.200650,4
2,1,3,0.428571,0.166667,0.608696,0.361288,6
3,1,4,0.071429,0.000000,0.318182,0.028732,1
4,1,5,0.071429,0.000000,0.400000,0.084649,1
...,...,...,...,...,...,...,...
11,52,11,0.133333,0.016190,0.428571,0.090940,2
12,52,12,0.133333,0.000000,0.384615,0.097801,2
13,52,13,0.133333,0.000000,0.394737,0.127715,2
14,52,14,0.400000,0.135556,0.535714,0.330243,6


In [51]:
mean_by_id = df.drop('week', axis=1).groupby('raccoon_ID').mean().reset_index()
for c in mean_by_id.columns.drop('raccoon_ID'):
    px.bar(mean_by_id, x="raccoon_ID", y=c).show()

In [64]:
mean_by_week = df.drop('raccoon_ID', axis=1).groupby('week').mean().reset_index().melt('week')
px.line(mean_by_week[mean_by_week.variable!='degree'], x='week', y='value', color='variable').show()
px.line(mean_by_week[mean_by_week.variable=='degree'], x='week', y='value', color='variable').show()

In [65]:
max_by_week = df.drop('raccoon_ID', axis=1).groupby('week').max().reset_index().melt('week')
px.line(max_by_week[max_by_week.variable!='degree'], x='week', y='value', color='variable').show()
px.line(max_by_week[max_by_week.variable=='degree'], x='week', y='value', color='variable').show()

In [67]:
min_by_week = df.drop('raccoon_ID', axis=1).groupby('week').min().reset_index().melt('week')
px.line(min_by_week[min_by_week.variable!='degree'], x='week', y='value', color='variable').show()
px.line(min_by_week[min_by_week.variable=='degree'], x='week', y='value', color='variable').show()

In [137]:
final_pos = pd.DataFrame()
filt = [str(x) for x in range(16)]
for f in files:
    g = nx.read_graphml(f[1]).subgraph(filt)
    final_pos = pd.concat([final_pos, pd.DataFrame(nx.spring_layout(nx.read_graphml(f[1]))).transpose()])
final_pos = final_pos.reset_index().groupby('index').mean()
final_pos = dict(final_pos.apply(lambda x: np.array([x[0], x[1]]), axis=1))

In [252]:
import plotly.graph_objects as go

def create_edges(G, pos):
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')
    return edge_trace

def weight(G, node):
    s = 0
    for g in G[node]:
        if g in filt:
            s += G[node][g]['weight']
    return np.log(s)

def create_nodes(G, pos):
    node_x = []
    node_y = []
    for p in pos.values():
        x, y = p
        node_x.append(x)
        node_y.append(y)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=20,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2))

    node_adjacencies = []
    node_text = []
    for k in pos.keys():
        if k in G.nodes:
            node_adjacencies.append(weight(G, k))
            node_text.append('Log-Interaction: '+str(weight(G, k)))
        else:
            node_adjacencies.append(0)
            node_text.append('Log-Interaction: '+str(0))

    node_trace.marker.color = node_adjacencies
    node_trace.text = node_text
    return node_trace

In [253]:
weeks = df['week'].unique()

# make figure
G = nx.read_graphml(files[0][1]).subgraph(filt)
fig_dict = {
    "data": [create_edges(G, final_pos), create_nodes(G, final_pos)],
    "layout": {},
    "frames": []
}

# fill in most of layout
fig_dict["layout"]['title'] = '<br>Evolution of connections through time'
fig_dict["layout"]['titlefont_size'] = 16
fig_dict["layout"]['showlegend'] = False
fig_dict["layout"]['hovermode'] = 'closest'
fig_dict["layout"]['margin'] = dict(b=20,l=5,r=5,t=40)
fig_dict["layout"]['xaxis'] = dict(showgrid=False, zeroline=False, showticklabels=False)
fig_dict["layout"]['yaxis'] = dict(showgrid=False, zeroline=False, showticklabels=False)

fig_dict["layout"]["sliders"] = {
    "args": [
        "transition", {
            "duration": 400,
            "easing": "cubic-in-out"
        }
    ],
    "initialValue": "1",
    "plotlycommand": "animate",
    "values": weeks,
    "visible": True
}
fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 1000, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Week:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

#Update the data and frames

for f in files:
    G = nx.read_graphml(f[1]).subgraph(filt)
    frame = {"data": [create_edges(G, final_pos), create_nodes(G, final_pos)], "name": f[0]}
    fig_dict["frames"].append(frame)
    slider_step = {"args": [
        [f[0]],
        {"frame": {"duration": 300, "redraw": False},
         "mode": "immediate",
         "transition": {"duration": 300}}
    ],
        "label": f[0],
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)
    
fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)

fig.show()